In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  #library to perform visualization in python
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import f1_score

ms_data=pd.read_csv('Microsoft_Data.csv') #command to read the dataset
ms_data['HasDetections']

#Step1 : Missing Values

#drop Functionality
def dropcolumn(drop_cols):
    ms_data.drop(columns=drop_cols,inplace=True)

# unique and identifier coulmns
def missingValues():
    missing_values_df=pd.DataFrame({'ColumnName':ms_data.columns,
                                'Missingcount':ms_data.isnull().sum(),
                                'MissingvaluePercentage':ms_data.isnull().sum()/ms_data.shape[0]*100
                           })
       
    #print(missing_values_df.sort_values(by='MissingvaluePercentage',ascending=False))
    return missing_values_df

missing_df=missingValues()

dp_col=[]
for row_index,row in missing_df.iterrows():
    if row[2]>30:
        dp_col.append(row[0])
    
dropcolumn(dp_col)
print(dp_col)
missingValues()


['DefaultBrowsersIdentifier', 'OrganizationIdentifier', 'PuaMode', 'SmartScreen', 'Census_ProcessorClass', 'Census_InternalBatteryType', 'Census_IsFlightingInternal', 'Census_ThresholdOptIn', 'Census_IsWIMBootEnabled']


,ColumnName,Missingcount,MissingvaluePercentage
MachineIdentifier,MachineIdentifier,0,0.000000
ProductName,ProductName,0,0.000000
EngineVersion,EngineVersion,0,0.000000
AppVersion,AppVersion,0,0.000000
AvSigVersion,AvSigVersion,0,0.000000
IsBeta,IsBeta,0,0.000000
RtpStateBitfield,RtpStateBitfield,196,0.345338
IsSxsPassiveMode,IsSxsPassiveMode,0,0.000000
AVProductStatesIdentifier,AVProductStatesIdentifier,222,0.391148
AVProductsInstalled,AVProductsInstalled,222,0.391148


In [146]:
ms_data.dtypes

#drop unique columns
drop_cols_unique=[col for col in ms_data.columns if (ms_data[col].nunique()==1 or 
                                                     ms_data[col].nunique()==ms_data.shape[0])]
print(drop_cols_unique)

dropcolumn(drop_cols_unique)

ms_data.shape



['MachineIdentifier', 'IsBeta', 'AutoSampleOptIn', 'Census_IsFlightsDisabled']


(56756, 70)

In [150]:
#fill missing values

for col in ms_data.columns:
    ms_data[col]=ms_data[col].fillna(ms_data[col].mode()[0])

ms_data.isnull().sum()

#convert data tpe to object

numeric_col=[col for col in ms_data.columns if ms_data[col].dtype!='object']
print(len(numeric_col))
for col in numeric_col:
    if(ms_data[col].nunique()==2):
        if((ms_data[col].unique()[0]==0 and ms_data[col].unique()[1]==1) or 
           (ms_data[col].unique()[1]==0 and ms_data[col].unique()[0]==1)):
            ms_data[col]=ms_data[col].astype('category')
    else:
        ms_data[col]=ms_data[col].astype('object')
        

ms_data.dtypes






45


ProductName                                            object
EngineVersion                                          object
AppVersion                                             object
AvSigVersion                                           object
RtpStateBitfield                                       object
IsSxsPassiveMode                                     category
AVProductStatesIdentifier                              object
AVProductsInstalled                                    object
AVProductsEnabled                                      object
HasTpm                                               category
CountryIdentifier                                      object
CityIdentifier                                         object
GeoNameIdentifier                                      object
LocaleEnglishNameIdentifier                            object
Platform                                               object
Processor                                              object
OsVer   

In [151]:
#onehotencoding and label encoding
binary_col=[col for col in ms_data.columns if (ms_data[col].nunique()==2 and ms_data[col].dtype=='object') ]
print(binary_col)

for col in binary_col:
    dummies_df=pd.get_dummies(ms_data[col],prefix=col,prefix_sep='_')  
    ms_data=pd.concat([ms_data,dummies_df],axis=1)
    ms_data.drop(columns=col,inplace=True)



print(ms_data.columns)
print(ms_data.shape)

['ProductName', 'Census_DeviceFamily']
Index(['EngineVersion', 'AppVersion', 'AvSigVersion', 'RtpStateBitfield',
       'IsSxsPassiveMode', 'AVProductStatesIdentifier', 'AVProductsInstalled',
       'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'CityIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease',
       'OsBuildLab', 'SkuEdition', 'IsProtected', 'SMode', 'IeVerIdentifier',
       'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity',
       'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM',
       'Census_ChassisTypeName',
       'Census_InternalPrimaryDiagonalDisplaySizeInInches'

In [155]:
ms_data['Processor'].value_counts()
ms_data['']

x64      51641
x86       5110
arm64        5
Name: Processor, dtype: int64

In [144]:
#test tran split (data split)
from sklearn.model_selection import train_test_split
y=ms_data['HasDetections']

ms_data.drop(columns='HasDetections',inplace=True)
ms_data.shape

X_train, X_test, y_train, y_test = train_test_split(ms_data, y, test_size=0.20, random_state=42)

X_train.shape
X_test.shape

(11352, 69)

['ProductName', 'Census_DeviceFamily']
Index(['EngineVersion', 'AppVersion', 'AvSigVersion', 'RtpStateBitfield',
       'IsSxsPassiveMode', 'AVProductStatesIdentifier', 'AVProductsInstalled',
       'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'CityIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform',
       'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease',
       'OsBuildLab', 'SkuEdition', 'IsProtected', 'SMode', 'IeVerIdentifier',
       'Firewall', 'UacLuaenable', 'Census_MDC2FormFactor',
       'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier',
       'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity',
       'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM',
       'Census_ChassisTypeName',
       'Census_InternalPrimaryDiagonalDisplaySizeInInches'

In [140]:
test_binary_col=[col for col in X_test.columns if (X_test[col].nunique()==2 and X_test[col].dtype=='object') ]
print(test_binary_col)

for col in test_binary_col:
    dummies_df=pd.get_dummies(X_test[col],prefix=col,prefix_sep='_')  
    X_test=pd.concat([X_test,dummies_df],axis=1)
    X_test.drop(columns=col,inplace=True)



print(X_test.columns)
print(X_test.shape)

['ProductName', 'Processor', 'Census_DeviceFamily', 'Census_OSArchitecture']
Index(['EngineVersion', 'AppVersion', 'AvSigVersion', 'RtpStateBitfield',
       'IsSxsPassiveMode', 'AVProductStatesIdentifier', 'AVProductsInstalled',
       'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'CityIdentifier',
       'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform', 'OsVer',
       'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'OsBuildLab',
       'SkuEdition', 'IsProtected', 'SMode', 'IeVerIdentifier', 'Firewall',
       'UacLuaenable', 'Census_MDC2FormFactor', 'Census_OEMNameIdentifier',
       'Census_OEMModelIdentifier', 'Census_ProcessorCoreCount',
       'Census_ProcessorManufacturerIdentifier',
       'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTotalCapacity',
       'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity',
       'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM',
       'Census_ChassisTypeName',
       'Census_InternalPrimaryDia